In [ ]:
#!pip install Levenshtein
#!pip install sentence_transformers

In [1]:
import intellikit as ik
import pandas as pd

In [2]:
# Define your DataFrame on which the recommendation system is going to be based
df = pd.DataFrame({
    'feature1': ['apple', 'orange', 'banana', 'grape'],
    'feature2': ['red', 'green', 'blue', 'yellow'],
    'feature3': ['small', 'large', 'medium', 'small'],
    'feature4': [1, 2, 3, 4],
    'feature5': [10, 20, 30, 40]
})

#Define your query 
query = pd.DataFrame({
    'feature1': ['apple'],
    'feature2': ['yellow'],
    'feature3': ['small'],
    'feature4': [3],
    'feature5': [30]

})

#Define you similarity calculation methods for your project
hamming = ik.sim_hamming
levenshtein = ik.sim_levenshtein
level = ik.sim_level
abs_diff = ik.sim_difference

# Assign the appro*priate similarity calculation functions to each feature
similarity_functions = {
    'feature1': hamming,
    'feature2': levenshtein,
    'feature3': level,
    'feature4': abs_diff,
    'feature5': abs_diff
}

In [3]:
unweighted_results = [similarity_functions[feature](df, query, feature) for feature in df.columns]

# Concatenate the resulting DataFrame columns along the rows
result = pd.concat(unweighted_results, axis=1)

print(result)
 

   feature1  feature2  feature3  feature4  feature5
0  1.000000  0.166667       1.0  0.333333  0.333333
1  0.166667  0.000000       0.0  0.666667  0.666667
2  0.166667  0.166667       0.5  1.000000  1.000000
3  0.200000  1.000000       1.0  0.750000  0.750000


In [4]:
#Applying the methods and weights and retrieving the top results
# Define weights for each feature
weights = {
    'feature1': 0.25,
    'feature2': 0.3,
    'feature3': 0.25,
    'feature4': 0.1,
    'feature5': 0.1
}

# Multiply each column by its corresponding weight
weighted_df = result * pd.Series(weights)

# Add a new column for the total of the weighted columns
weighted_df['weighted_total'] = weighted_df.sum(axis=1)

print(weighted_df)

   feature1  feature2  feature3  feature4  feature5  weighted_total
0  0.250000      0.05     0.250  0.033333  0.033333        0.616667
1  0.041667      0.00     0.000  0.066667  0.066667        0.175000
2  0.041667      0.05     0.125  0.100000  0.100000        0.416667
3  0.050000      0.30     0.250  0.075000  0.075000        0.750000


In [5]:
#You then go ahead and sort by the weighted total and retrieved the top k ranked cases